In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
os.chdir('/content/drive/My Drive/Pose Estimation/orvee/Pretrained openpose')

Mounted at /content/drive


In [ ]:
from __future__ import division
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import Model, Sequential, utils
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model

import pandas as pd 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D, concatenate, Input, Permute, Concatenate, BatchNormalization
from keras import applications
#from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam  
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import pickle
import random
import cv2

In [ ]:
def get_hand_model(modelpath):   #modelpath = hand_models

    prototxt   = os.path.join(modelpath, "pose_deploy.prototxt")
    caffemodel = os.path.join(modelpath, "pose_iter_102000.caffemodel")
    hand_model = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)

    return hand_model

num_points = 21
point_pairs = [[0,1],[1,2],[2,3],[3,4],
                    [0,5],[5,6],[6,7],[7,8],
                    [0,9],[9,10],[10,11],[11,12],
                    [0,13],[13,14],[14,15],[15,16],
                    [0,17],[17,18],[18,19],[19,20]]
#inWidth = 64
inHeight = 368
threshold = 0.1
modelpath = "hand_models"
hand_net = get_hand_model(modelpath)

def pose_predict(imgfile):
        img_cv2 = cv2.imread(imgfile)
        # img_height, img_width, _ = img_cv2.shape
        img_height, img_width, _ = 224, 224, 1
        aspect_ratio = img_width / img_height

        inWidth = int(((aspect_ratio * inHeight) * 8) // 8)
        inpBlob = cv2.dnn.blobFromImage(img_cv2, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

        hand_net.setInput(inpBlob)

        output = hand_net.forward()

        # # vis heatmaps
        # vis_heatmaps(imgfile, output)

        #
        points = []
        for idx in range(num_points):
            probMap = output[0, idx, :, :] # confidence map.
            probMap = cv2.resize(probMap, (img_width, img_height))

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            if prob > threshold:
                points.append([int(point[0]), int(point[1])])
            else:
                points.append([0, 0])

        return points

In [ ]:
#################TO MAKE NPY FILE FROM THE DATASET, JUST CHANGE path to train and test directory one by one
imgs_path = "bangla_dataset/RESIZED_TESTING_DATA/"
img_files = os.listdir(imgs_path)
modelpath = "hand_models"
pose_list =[]
img_list = []
label_list = []


#print(img_files)
for img_file in img_files:
  pictures_dir = os.path.join(imgs_path, img_file)
  print(pictures_dir)
  pictures = [os.path.join(pictures_dir, pic) for pic in os.listdir(pictures_dir)]
  for pic in pictures:
    # img = np.array(Image.open(pic).resize((64,64)))
    # img_list.append(img)
    img = cv2.imread(pic,0)
    pic_pose = cv2.resize(img, (224, 224))
    img = np.array(pic_pose)
    img = img.astype('float32')/255
    img_list.append(img)
    res_points = pose_predict(pic)
    pose_list.append(res_points)
    label = tf.keras.utils.to_categorical(img_file, num_classes=38)
    label_list.append(label)            
print("[INFO]Done.")
 

bangla_dataset/RESIZED_TESTING_DATA/0
bangla_dataset/RESIZED_TESTING_DATA/1
bangla_dataset/RESIZED_TESTING_DATA/10
bangla_dataset/RESIZED_TESTING_DATA/11
bangla_dataset/RESIZED_TESTING_DATA/12
bangla_dataset/RESIZED_TESTING_DATA/13
bangla_dataset/RESIZED_TESTING_DATA/14
bangla_dataset/RESIZED_TESTING_DATA/15
bangla_dataset/RESIZED_TESTING_DATA/16
bangla_dataset/RESIZED_TESTING_DATA/17
bangla_dataset/RESIZED_TESTING_DATA/18
bangla_dataset/RESIZED_TESTING_DATA/19
bangla_dataset/RESIZED_TESTING_DATA/2
bangla_dataset/RESIZED_TESTING_DATA/20
bangla_dataset/RESIZED_TESTING_DATA/21
bangla_dataset/RESIZED_TESTING_DATA/22
bangla_dataset/RESIZED_TESTING_DATA/23
bangla_dataset/RESIZED_TESTING_DATA/24
bangla_dataset/RESIZED_TESTING_DATA/25
bangla_dataset/RESIZED_TESTING_DATA/26
bangla_dataset/RESIZED_TESTING_DATA/27
bangla_dataset/RESIZED_TESTING_DATA/28
bangla_dataset/RESIZED_TESTING_DATA/29
bangla_dataset/RESIZED_TESTING_DATA/3
bangla_dataset/RESIZED_TESTING_DATA/30
bangla_dataset/RESIZED_TESTIN

NotADirectoryError: ignored

In [ ]:
#Saving the training numpy files to feed to the main neural network. Similarly do it for test images.
np.save("bangla_dataset/np_files/imglist_t.npy", img_list)   #npy contains (no. of images, img_length, img_breadth, no. of channel)
np.save("bangla_dataset/np_files/poselist_t.npy", pose_list)   #npy contains (no. of images, no. of points = 22)
np.save("bangla_dataset/np_files/labellist_t.npy", label_list)  #npy contains (no. of images, no. of labels = 38)

In [ ]:
#Example of loading a numpy file for use
a = np.load("bangla_dataset/np_files/labellist_t.npy", allow_pickle= True)
a.shape

(1520, 38)